In [1]:
import numpy as np
from estimator import BalancingWeightsEstimator

## Kang Schafer

In [2]:
class kang_schafer:
    def __init__(self, size: int = 2000):
        self.size = size
        np.random.seed(123)  # Added for reproducibility in this test run
        self.covariates = np.random.randn(size, 4)
        propensity_score = 1.0 / (
            1.0 + np.exp(-np.dot(self.covariates, np.array([-1.0, 0.5, -0.25, -0.1])))
        )
        self.treatment = np.random.binomial(1, propensity_score)
        self.outcome = (
            210.0
            + np.dot(self.covariates, np.array([27.4, 13.7, 13.7, 13.7]))
            + np.random.randn(size)
        )

    @property
    def transformed_covariates(self) -> np.ndarray:
        x1, x2, x3, x4 = np.hsplit(self.covariates, 4)
        return np.hstack(
            [
                np.exp(x1 / 2.0),
                x2 / (1 + np.exp(x1)) + 10.0,
                np.power(x1 * x3 / 25 + 0.6, 3),
                np.square(x2 + x4 + 20.0),
            ]
        )

In [3]:
# --- Test on Kang-Schafer data ---
ks_data = kang_schafer(size=10_000)

### Correctly Specified

In [4]:
Y_t_raw = ks_data.outcome[ks_data.treatment == 1]
Y_c_raw = ks_data.outcome[ks_data.treatment == 0]
naive_diff_raw = np.mean(Y_t_raw) - np.mean(Y_c_raw)
print(f"\nNaive Difference in Means (Raw Covariates): {naive_diff_raw:.4f}")
print("True ATT should be 0.")


Naive Difference in Means (Raw Covariates): -20.1077
True ATT should be 0.


In [5]:
print("--- Testing with RAW COVARIATES ---")
estimator_raw = BalancingWeightsEstimator(solver_iters=20000, solver_tolerance=1e-10)
estimator_raw.fit(ks_data.covariates, ks_data.outcome, ks_data.treatment)
estimator_raw.summary()

--- Testing with RAW COVARIATES ---
--- Balancing Weights Estimator Summary ---
Estimated ATT: -0.0375
Mean Y treated: 200.4843
Weighted Mean Y control: 200.5218
Number of treated units: 5021
Number of control units: 4979


In [6]:
estimator_raw.balance_table()

{'Treated Mean': array([-0.3888113 ,  0.20242872, -0.09164977, -0.0277883 ]),
 'Control Mean': array([ 0.40614924, -0.19347029,  0.11011067,  0.04333858]),
 'Weighted Control Mean': array([-0.38879409,  0.20242316, -0.09164405, -0.02778622]),
 'Pre-weighting Imbalance': array([-0.79496055,  0.39589901, -0.20176044, -0.07112689]),
 'Post-weighting Imbalance': array([-1.72078585e-05,  5.56851491e-06, -5.72105360e-06, -2.08195967e-06])}

### misspecification

In [7]:
estimator_transformed = BalancingWeightsEstimator(
    solver_iters=20000, solver_tolerance=1e-10
)
estimator_transformed.fit(
    ks_data.transformed_covariates, ks_data.outcome, ks_data.treatment
)
estimator_transformed.summary()

--- Balancing Weights Estimator Summary ---
Estimated ATT: -24.4460
Mean Y treated: 200.4843
Weighted Mean Y control: 224.9303
Number of treated units: 5021
Number of control units: 4979


#### polynomials

In [12]:
from sklearn.preprocessing import PolynomialFeatures

# Assuming ks_data is your Kang-Schafer data instance
original_transformed_cov = ks_data.transformed_covariates
poly = PolynomialFeatures(degree=4, include_bias=False, interaction_only=False)
# include_bias=False is generally good as simplex weights sum to 1.
# interaction_only=False will include X1^2, X2^2 in addition to X1*X2 etc.
X_poly_for_estimator = poly.fit_transform(original_transformed_cov)

In [13]:
estimator_transformed.fit(X_poly_for_estimator, ks_data.outcome, ks_data.treatment)
estimator_transformed.summary()

/home/alal/Desktop/code/econometrics/balancer/estimator.py:125: RuntimeWarning: overflow encountered in exp
  beta = beta * np.exp(-up)
/home/alal/Desktop/code/econometrics/balancer/estimator.py:134: RuntimeWarning: invalid value encountered in divide
  beta_section / sum_beta_section


--- Balancing Weights Estimator Summary ---
Estimated ATT: -20.1077
Mean Y treated: 200.4843
Weighted Mean Y control: 220.5920
Number of treated units: 5021
Number of control units: 4979


#### Kernel approximation

In [10]:
from sklearn.kernel_approximation import Nystroem

original_transformed_cov = ks_data.transformed_covariates
# Choose kernel, gamma, n_components. These may require tuning.
# For RBF kernel, gamma is crucial. A common heuristic is 1 / (n_features * X.var())
# or you can cross-validate.
n_features_orig = original_transformed_cov.shape[1]
gamma_rbf = 1.0 / (n_features_orig * np.var(original_transformed_cov))

nystroem_approx = Nystroem(
    kernel="rbf", gamma=gamma_rbf, random_state=42, n_components=100
)  # Number of components to approximate

X_nystroem_for_estimator = nystroem_approx.fit_transform(original_transformed_cov)

In [11]:
estimator_transformed_nystroem = BalancingWeightsEstimator(
    solver_iters=20000, solver_tolerance=1e-10
)
estimator_transformed_nystroem.fit(
    X_nystroem_for_estimator,
    ks_data.outcome,
    ks_data.treatment
)
estimator_transformed_nystroem.summary()

--- Balancing Weights Estimator Summary ---
Estimated ATT: -22.4781
Mean Y treated: 200.4843
Weighted Mean Y control: 222.9625
Number of treated units: 5021
Number of control units: 4979
